In [31]:
import zipfile, os
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Imputer
from sklearn.model_selection import GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from functions.functions import previous_yrs, opponent_stats

# Read in files

In [2]:
_file = os.path.join('Data', 'finalgamedata.zip')
zf = zipfile.ZipFile(_file)
gamestats = pd.read_csv(zf.open('gamestats.csv'))
gamelist = pd.read_csv(zf.open('gameslist.csv'))
gamelist['opponentName'] = gamelist['opponentName'].str.replace("  ", " ")

In [3]:
gamestats.TOP = pd.to_timedelta(gamestats.TOP).astype('timedelta64[s]')

In [4]:
# Create a list of the teams 
teamlist = list(gamelist['Team'].unique())

In [5]:
# Create the opponent's game list data for use later 
opponent_game_list_data = gamelist.copy(deep=True)
opponent_game_list_data.drop(['Opponent', 'Result', 'year', 'opponentName'], axis = 1, inplace = True)
opponent_game_list_data.rename(columns={'Team':'opponentName'}, inplace = True)
# Relabel the variables
opponent_game_list_columns = ['opp_'+ x if x not in ['Date', 'opponentName']
                              else x
                              for x in list(opponent_game_list_data.columns) 
                             ]

opponent_game_list_data.columns = opponent_game_list_columns

In [6]:
gamelist = gamelist.merge(opponent_game_list_data, 
                          left_on = ['opponentName', 'Date'],
                          right_on = ['opponentName', 'Date'], 
                          how = 'left'
                         );

# Target variables

In [7]:
target_variables = ["FG","RushNetYards","PassYards","Interceptions","Pct", "RZScores",
                    "Punts","FumblesLost","Penalties","Plays","def_RushNetYards","def_PassYards",
                    "def_Interceptions","def_Punts", "def_FF","def_PenaltyYards","def_PDef","Sacks","Tackles",
                    "def_Plays", "TOP"
                   ]

# Create opponent variable names

In [8]:
# Create the opponent variable names 
opponent_variables = ['opp_' + x for x in target_variables]

# Create dataset

In [9]:
base_year_mask = ~((gamelist.year == 2013) & (gamelist['gamenumber'] == 1))

In [10]:
gamelist[target_variables] = gamelist[base_year_mask].apply(lambda x: previous_yrs(team = x['Team'], 
                                                                                   year = x['year'], 
                                                                                   game = x['gamenumber'], 
                                                                                   cols = target_variables, 
                                                                                   gamestats = gamestats,
                                                                                   debug = False
                                                                                  ), 
                                                            axis = 1
                                                           )

In [11]:
# Subset the datset to only the needed games 
games = gamelist[base_year_mask].copy()

# Calculate the opponent game data 
games[opponent_variables] = gamelist.apply(lambda x: opponent_stats(team = x['Team'],
                                           date = x['Date'], 
                                           year = x['year'], 
                                           cols = target_variables,
                                           gamestats = gamestats,
                                           debug = False
                                          )
                             if x['opponentName'] in teamlist
                             else pd.Series([None for x in range(0, len(target_variables))]),
                             axis = 1
                            )

# Game list file variables to include

In [12]:
game_list_variables = ['gamenumber', 'Coach_years', 'Coach_wins', 'HomeAway', 'history_WL_1yrs']
game_list_variables = game_list_variables + ['opp_' + x for x in game_list_variables]

In [13]:
final_variables = target_variables + opponent_variables + game_list_variables

In [14]:
games[final_variables];

# Count the number of missing variables

In [15]:
games['missing'] = games[final_variables].isnull().sum(axis=1)

In [16]:
games[final_variables];

# Create variables for analysis 

In [17]:
y = games[games['missing'] == 0]['WinLoss'].values
X = games[games['missing'] == 0][final_variables].values

# Run single Random Forest Classifier

In [18]:
# Create the train test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [19]:
# Create and fit the DT Classifier 
dt = RandomForestClassifier(random_state = 42)
dt.fit(X= X_train, y=y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [20]:
print("Accuracy:\t", (y_train == dt.predict(X_train)).mean())
print("Accuracy:\t", (y_test == dt.predict(X_test)).mean())

Accuracy:	 0.99292324442
Accuracy:	 0.71412803532


# Run Grid Search on Random Forest Classifier

In [21]:
imputer = Imputer()
forest = RandomForestClassifier()
pipe = Pipeline(steps=[('imputer', imputer), ('forest', forest)])

In [33]:
grid_parameters = {'forest__criterion': ['gini', 'entropy'], 
                     'forest__max_depth':[1,2,3,4,5], 
                     'forest__n_estimators':[100, 500, 1000, 1250], 
                     'forest__n_jobs':[-1], 
                     'forest__random_state':[42], 
                     'forest__min_samples_split':[5, 10],
                     'imputer__missing_values':['NaN'], 
                     'imputer__strategy':['mean'], 
                     'imputer__axis':[0], 
                    }

In [34]:
random_grid_parameters = {'forest__criterion': ['gini', 'entropy'], 
                            'forest__max_depth':[1,5], 
                            'forest__n_estimators':[100, 1250], 
                            'forest__n_jobs':[-1], 
                            'forest__random_state':[42], 
                            'forest__min_samples_split':[5, 10],
                            'imputer__missing_values':['NaN'], 
                            'imputer__strategy':['mean'], 
                            'imputer__axis':[0], 
                           }

In [23]:
grid = GridSearchCV(pipe, grid_parameters, cv=3)
grid.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('forest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_im...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'forest__criterion': ['gini', 'entropy'], 'forest__max_depth': [1, 2, 3, 4, 5], 'forest__n_estimators': [100, 500, 1000, 1250], 'forest__n_jobs': [-1], 'forest__random_state': [42], 'forest__min_samples_split': [5, 10], 'imputer__missing_values': ['NaN'], 'imputer__strategy': ['mean'], 'imputer__axis': [0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [35]:
grid = RandomizedSearchCV(pipe, random_grid_parameters, cv=3)
grid.fit(X_train, y_train)

RandomizedSearchCV(cv=3, error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('forest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_im...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'forest__criterion': ['gini', 'entropy'], 'forest__max_depth': [1, 5], 'forest__n_estimators': [100, 1250], 'forest__n_jobs': [-1], 'forest__random_state': [42], 'forest__min_samples_split': [5, 10], 'imputer__missing_values': ['NaN'], 'imputer__strategy': ['mean'], 'imputer__axis': [0]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbo

In [36]:
tree_model = grid.best_estimator_
print("Accuracy:\t", grid.best_score_) 
print("Best Parameters:\t", grid.best_params_) 

Accuracy:	 0.74251497006
Best Parameters:	 {'imputer__strategy': 'mean', 'imputer__missing_values': 'NaN', 'imputer__axis': 0, 'forest__random_state': 42, 'forest__n_jobs': -1, 'forest__n_estimators': 100, 'forest__min_samples_split': 5, 'forest__max_depth': 5, 'forest__criterion': 'gini'}


In [37]:
feature_importance = grid.best_estimator_.named_steps['forest'].feature_importances_

In [38]:
x = list(zip(final_variables, feature_importance))

In [39]:
importance = pd.DataFrame(x, columns=['var', 'importance'])

In [40]:
importance.sort_values('importance', ascending = False)

,var,importance
51,opp_history_WL_1yrs,0.189242
46,history_WL_1yrs,0.139044
49,opp_Coach_wins,0.067623
31,opp_def_RushNetYards,0.054706
5,RZScores,0.036813
10,def_RushNetYards,0.036099
26,opp_RZScores,0.035753
44,Coach_wins,0.030868
27,opp_Punts,0.030461
25,opp_Pct,0.029029


In [41]:
grid.score(X_test, y_test)

0.7527593818984547